# Julia 深度學習：卷積神經網路模型簡介

本範例有可選用套件 CuArrays，請在執行以下範例前先安裝。

```
] add CuArrays
```

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

## 載入資料

In [2]:
train_X, train_y = MNIST.traindata(Float32);
test_X, test_y = MNIST.testdata(Float32);

In [3]:
train_X = reshape(train_X, 28, 28, 1, :);
test_X = reshape(test_X, 28, 28, 1, :);
train_y = onehotbatch(train_y, 0:9);
test_y = onehotbatch(test_y, 0:9);

In [4]:
batchsize = 1024;
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true);
test = DataLoader(test_X, test_y, batchsize=batchsize);

## CNN 模型

In [5]:
model = Chain(
    Conv((3, 3), 1=>16, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 16=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32=>32, pad=(1,1), relu),
    MaxPool((2,2)),
    flatten,
    Dense(288, 10),
    softmax)

Chain(Conv((3, 3), 1=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 16=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(288, 10), softmax)

## 使用 CUDA

In [ ]:
using CuArrays
model = model |> gpu
train_X = train_X |> gpu
train_y = train_y |> gpu
test_X = test_X |> gpu
test_y = test_y |> gpu

In [ ]:
CuArrays.allowscalar(false)

## 損失函數

In [6]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

## Callback 函式

In [7]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l/length(test)
end

test_loss (generic function with 1 method)

In [8]:
evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

## 模型訓練

In [9]:
epochs = 20
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.295753f0
test_loss() = 1.9420303f0
test_loss() = 1.7135065f0
test_loss() = 1.6613283f0
test_loss() = 1.6287315f0
test_loss() = 1.6232039f0
test_loss() = 1.6071837f0


┌ Info: Epoch 2
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.8957886f0
test_loss() = 1.7022787f0
test_loss() = 1.6681086f0
test_loss() = 1.6249506f0
test_loss() = 1.6031818f0
test_loss() = 1.5935194f0
test_loss() = 1.5852354f0


┌ Info: Epoch 3
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

test_loss() = 1.5965443f0
test_loss() = 1.5827436f0
test_loss() = 1.5701668f0
test_loss() = 1.5675156f0
test_loss() = 1.5673304f0
test_loss() = 1.5668051f0
test_loss() = 1.565996f0


┌ Info: Epoch 5
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.5708936f0
test_loss() = 1.5667822f0
test_loss() = 1.5649023f0
test_loss() = 1.565459f0
test_loss() = 1.5640906f0
test_loss() = 1.5653436f0
test_loss() = 1.5628531f0


┌ Info: Epoch 6
└ @ Main C:\Users\qwerz\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


InterruptException: InterruptException:

## 模型評估

In [10]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [11]:
accuracy(test_X, test_y)

0.8961